In [9]:
import os, yaml
import numpy as np
import pandas as pd
import pprint
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
import spotipy
from dotenv import load_dotenv
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
import tiktoken
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [18]:
#Llama completion model
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path='/Users/kpathak/Downloads/openbuddy-llama2-13b-v11.1.Q4_0.gguf',
    temperature=0,
    n_ctx=4095,
    max_tokens=4095,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,
)
COMPLETIONS_MODEL=llm

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/kpathak/Downloads/openbuddy-llama2-13b-v11.1.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 37632,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q4_0 

In [10]:
#Collecting openai and soptify OpenAPI Spec
!wget https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
!mv openapi.yaml openai_openapi.yaml
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml

--2023-09-16 19:11:56--  https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152811 (149K) [text/plain]
Saving to: 'openapi.yaml'

openapi.yaml        100%[===================>] 149.23K  --.-KB/s    in 0.09s   

2023-09-16 19:11:56 (1.67 MB/s) - 'openapi.yaml' saved [152811/152811]

--2023-09-16 19:11:56--  https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2893

In [11]:
#Getting reducde open api spec needed for efficent processing
with open("openai_openapi.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

In [13]:
#spotfiy credetials
load_dotenv('/Users/kpathak/Library/CloudStorage/OneDrive-ManhattanAssociates/Learnings/api_key.env')
SPOTIPY_CLIENT_ID=os.environ.get('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET=os.environ.get('SPOTIPY_CLIENT_SECRET')
SPOTIPY_REDIRECT_URI=os.environ.get('SPOTIPY_REDIRECT_URI')
print(SPOTIPY_REDIRECT_URI)

http://localhost:8088/callback/


In [14]:
#Getting your credential for your app get from below link
#https://developer.spotify.com/documentation/web-api/concepts/authorization
import spotipy.util as util
from langchain.requests import RequestsWrapper


def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(
        raw_spec["components"]["securitySchemes"]["oauth_2_0"]["flows"][
            "authorizationCode"
        ]["scopes"].keys()
    )
    access_token = util.prompt_for_user_token(scope=",".join(scopes))
    return {"Authorization": f"Bearer {access_token}"}


# Get API credentials.
headers = construct_spotify_auth_headers(raw_spotify_api_spec)
requests_wrapper = RequestsWrapper(headers=headers)

In [15]:
#Spec length
endpoints = [
    (route, operation)
    for route, operations in raw_spotify_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

63

In [16]:
import tiktoken

enc = tiktoken.encoding_for_model("text-davinci-003")

def count_tokens(s):
    return len(enc.encode(s))


count_tokens(yaml.dump(raw_spotify_api_spec))

81143

In [19]:
#Testing with examples
#Making user query action done through the agent
#Here user wants to create a splotify playlist for song blue

spotify_agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, COMPLETIONS_MODEL)
user_query = (
    "make me a playlist with the first song from kind of blue. call it  Blues."
)
spotify_agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to create a new playlist with the first song from Kind of Blue called "Blues"Action: api_planner
Action Input: I need to find the right API calls to create a new playlist with the first song from Kind of Blue called "Blues"


llama_print_timings:        load time =  4316.80 ms
llama_print_timings:      sample time =    42.07 ms /    42 runs   (    1.00 ms per token,   998.41 tokens per second)
llama_print_timings: prompt eval time = 268081.53 ms /   581 tokens (  461.41 ms per token,     2.17 tokens per second)
llama_print_timings:        eval time = 20135.88 ms /    41 runs   (  491.12 ms per token,     2.04 tokens per second)
llama_print_timings:       total time = 288546.42 ms
Llama.generate: prefix-match hit


 1. GET /me/albums to get the album's ID
2. GET /me/tracks/{id} to get the track's ID
3. POST /users/{user_id}/playlists to create a new playlist for the user
4. DELETE /users/{user_id}/playlists/{playlist_id}/tracks/{track_id} to remove the first song from the playlist
5. GET /me/playlists/{playlist_id}/tracks to get the updated list of tracks in the playlist

Observation:  1. GET /me/albums to get the album's ID
2. GET /me/tracks/{id} to get the track's ID
3. POST /users/{user_id}/playlists to create a new playlist for the user
4. DELETE /users/{user_id}/playlists/{playlist_id}/tracks/{track_id} to remove the first song from the playlist
5. GET /me/playlists/{playlist_id}/tracks to get the updated list of tracks in the playlist

Thought:


llama_print_timings:        load time =  4316.80 ms
llama_print_timings:      sample time =   122.65 ms /   123 runs   (    1.00 ms per token,  1002.85 tokens per second)
llama_print_timings: prompt eval time = 1639349.88 ms /  3180 tokens (  515.52 ms per token,     1.94 tokens per second)
llama_print_timings:        eval time = 75171.99 ms /   122 runs   (  616.16 ms per token,     1.62 tokens per second)
llama_print_timings:       total time = 1715886.42 ms
Llama.generate: prefix-match hit


 I'm ready to execute the plan.
Action: api_controller
Action Input: 1) GET /me/albums to get the album's ID
2) GET /me/tracks/{id} to get the track's ID
3) POST /users/{user_id}/playlists to create a new playlist for the user
4. DELETE /users/{user_id}/playlists/{playlist_id}/tracks/{track_id} to remove the first song from the playlist
5. GET /me/playlists/{playlist_id}/tracks to get the updated list of tracks in the playlist
...
 I'm ready to execute the plan.
Action: api_controller
Action Input: 1) GET /me/albums to get the album's ID
2) GET /me/tracks/{id} to get the track's ID
3) POST /users/{user_id}/playlists to create a new playlist for the user
4. DELETE /users/{user_id}/playlists/{playlist_id}/tracks/{track_id} to remove the first song from the playlist
5. GET /me/playlists/{playlist_id}/tracks to get the updated list of tracks in the playlist
...



llama_print_timings:        load time =  4316.80 ms
llama_print_timings:      sample time =   143.35 ms /   144 runs   (    1.00 ms per token,  1004.52 tokens per second)
llama_print_timings: prompt eval time = 343372.26 ms /   746 tokens (  460.28 ms per token,     2.17 tokens per second)
llama_print_timings:        eval time = 71402.94 ms /   143 runs   (  499.32 ms per token,     2.00 tokens per second)
llama_print_timings:       total time = 415561.37 ms


ValueError: GET /me/tracks/{id} endpoint does not exist.